In [ ]:
pip install azure-eventhub 


In [ ]:
import requests
import json
import time
from datetime import datetime
from datetime import datetime, timezone, timedelta
from azure.eventhub import EventHubProducerClient, EventData
import notebookutils



# -------------------------------
# Configuration
# -------------------------------
API_KEY = "b97479a794fa4c94b9c64911252710"
IST = timezone(timedelta(hours=5, minutes=30))
points = {
    "Connaught Place": (28.6139, 77.2090),
    "Noida": (28.5355, 77.3910),
    "West Delhi": (28.7041, 77.1025),
    "Gurugram": (28.4595, 77.0266),
    "New Delhi":(28.633,77.222)
}
#WEATHER_URL = f"https://api.weatherapi.com/v1/current.json?q={LAT},{LON}&key={API_KEY}&aqi=yes"

EVENTHUB_CONNECTION_STR = "Endpoint=sb://esehbn3ju5fjl3h70565s1.servicebus.windows.net/;SharedAccessKeyName=key_86387cfa-aa72-4f7b-b100-4d9f4d74b492;SharedAccessKey=l+PHgLkXVTWxq2YLSTyMhwbYExN7/ODnn+AEhPqZZmo=;EntityPath=es_b948d01c-e76a-4c65-81cc-7363f24af34f"
EVENTHUB_NAME = "weatherapi"
producer = EventHubProducerClient.from_connection_string(conn_str=EVENTHUB_CONNECTION_STR)

# Get the current datetime object
now = datetime.now()
# Convert to Unix timestamp (float, seconds since epoch)
timestamp_seconds = now.timestamp()
# Convert to milliseconds and cast to an integer
timestamp_millis = int(timestamp_seconds * 1000)
print(timestamp_millis)
dt_str = datetime.now(IST).isoformat()
dt_obj = datetime.fromisoformat(dt_str)
formatted_dt = dt_obj.strftime("%Y-%m-%d %H:%M")
#run_id = mssparkutils.pipeline.getPipelineRunId()
# -------------------------------
# Fetch weather data
# -------------------------------
def fetch_weather(location_name, lat, lon):
    url = f"https://api.weatherapi.com/v1/current.json?q={lat},{lon}&key={API_KEY}&aqi=yes"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Flatten JSON to match schema
        event = {
            "geo_point_name": location_name,
            "location_name": data["location"]["name"],
            "location_region": data["location"]["region"],
            "location_country": data["location"]["country"],
            "location_lat": data["location"]["lat"],
            "location_lon": data["location"]["lon"],
            "location_tz_id": data["location"]["tz_id"],
            "localtime":formatted_dt,
            "temp_c": data["current"]["temp_c"],
            "condition_text": data["current"]["condition"]["text"],
            "wind_kph": data["current"]["wind_kph"],
            "humidity": data["current"]["humidity"],
            "air_quality_co": data["current"]["air_quality"]["co"],
            "air_quality_no2": data["current"]["air_quality"]["no2"],
            "air_quality_pm25": data["current"]["air_quality"]["pm2_5"],
            "air_quality_pm10": data["current"]["air_quality"]["pm10"],
            "dateTime": formatted_dt,          
            "timestamp_millis": timestamp_millis
           
        }
        print(f"✅ Fetched weather for {location_name} ({lat}, {lon})")
        
        return event
    except requests.RequestException as e:
        print(f"❌ Error fetching weather for {location_name}: {e}")
        return None

# -------------------------------
# Send batch of events to Event Hub
# -------------------------------
def send_to_eventhub(events):
    try:
        with producer:
            event_data_batch = producer.create_batch()
            for event in events:
                event_data_batch.add(EventData(json.dumps(event)))
            producer.send_batch(event_data_batch)
        print(f"📤 Sent {len(events)} events to Event Hub at {datetime.utcnow().isoformat()}")
    except Exception as e:
        print(f"❌ Error sending to Event Hub: {e}")

all_events = []

for name, (lat, lon) in points.items():
        event = fetch_weather(name, lat, lon)
        if event:
            all_events.append(event)
            time.sleep(2)  # small delay between API calls to avoid throttling
if all_events:
        send_to_eventhub(all_events)
        print(all_events)

  
